<h1><a id='outline'>Project Outline</a></h1>

* [Sequencing Summaries](#seqSums)
    *  [Adapter Trimming and QC (Trimmomatic) Report](#readbd)
    *  [Host Plant Alignment Report](#pAlign)
    *  [Switchgrass Fungal Alignments](#swAlign)
    *  [Alignment Conclusions](#aConc)
* [Annotation Report](#anno)   


* [NMDS of annotated contigs](#nmds)
  *  [All Crops](#anmds)
  *  [Switchgrass](#sgnmds)
  *  [Miscanthus](#msnmds)
*  [Function Analysis](#metaT)

In [1]:
#HIDDEN
%matplotlib inline
from glob import glob
from ipywidgets import Text, Dropdown, Output, interact
from IPython.display import Audio
from pandas import DataFrame, merge, read_csv, Series, to_datetime
from pickle import dump, load
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from random import randint 
from sklearn import manifold
from sklearn.metrics import euclidean_distances
import numpy as np
import warnings

done = Audio('done.wav',autoplay=True)
warnings.simplefilter(action='ignore', category=FutureWarning)
def comma(number): return "{:,}".format(number)
def scolor(i,n,pickColor): 
    scalar = 255 - int((i/float(n))*120) 
    if pickColor: return '#%02X%02X%02X' % (128,scalar,128)
    return '#%02X%02X%02X' % (75,scalar,200)
global metadataG
# For metaG
metadataG = read_csv("metadata/GLBRC_MetaG_Metadata.tsv",sep='\t')
metadataG.set_index("nucleic_acid_name",inplace=True)
metadataG.drop(["source","sampling ID","sequencing_type","height_mean_cm","air_temp_c","rep","Sampling Time","reads_file_name","treatment"],axis=1,inplace=True) 

for id in metadataG.index: metadataG.loc[id,"type"] = metadataG[metadataG.index == id].plot_name[0][0:2]
metadataG['Date'] = to_datetime(metadataG.sampling_date)
metadataG=metadataG.sort_values(by=["type","Date"])
metadataG.drop(["sampling_date"],axis=1,inplace=True)
print(metadataG.shape)

metaG_Reads = read_csv("mapping/metaG/hostRemovalFlagstats/multiqc_data/multiqc_bowtie2.txt",sep="\t")
for id in metaG_Reads.index: metaG_Reads.at[id,"Sample"] = metaG_Reads.at[id,"Sample"].replace(".stat","")
metaG_Reads.set_index("Sample",inplace=True)
metadataG = merge(metadataG,metaG_Reads,left_index=True,right_index=True)
metadataG.sort_values("Date",inplace=True)

# metadataG.head(3)
@interact
def show_articles_more_than(column=['total_reads', 'paired_total', 'paired_aligned_none', 'paired_aligned_one'], 
                            x=(0, 20000000, 50000)):
    return metadataG.loc[metadataG[column] > x]

(136, 3)


interactive(children=(Dropdown(description='column', options=('total_reads', 'paired_total', 'paired_aligned_n…

In [2]:
# nbi:hide_in
import ipywidgets
from ipywidgets import Text, Dropdown, Output, interact  # If not already imported
#AppLayout
output_radio_selected = Text() # Used to take the user input and access it when needed
displayType = Dropdown(
    options=["Percentage","Count"],
    value='Percentage',
    description='Reads by:',
    disabled=False,
) # nbid:hide_ind remove ds
# radio_input = widgets.RadioButtons(options=['Option 1', 'Option 2']) # Declare the set of radio buttons and provide options
output = Output()

@output.capture()
def on_change_displayType(sender): # Connect the input from the user to the output so we can access it
    output.clear_output()
#     print(sender)
    global selected_option # Global variable to hold the user input for reuse in your code
    output_radio_selected.value = displayType.value
    selected_option = output_radio_selected.value # Example variable assigned the selected value
    print('Selected option set to: ' + selected_option) # For test purposes

displayType.observe(on_change_displayType, names=['value']) # Run the bind... function when the radio button is changed
displayType # Display the radio buttons to the user
display(displayType)
display(output)

Dropdown(description='Reads by:', options=('Percentage', 'Count'), value='Percentage')

Output()

In [3]:
#HIDDEN
def square(x):
    return x * x
interact(square, x=(0, 100, 10));

interactive(children=(IntSlider(value=50, description='x', step=10), Output()), _dom_classes=('widget-interact…

# Hello world2

<h1><a id="seqSums">Sequencing Summaries</a></h1>

### What is the quality of the WGS metagenomic reads?

<h4><a id="readbd">Adapter Trimming and QC (Trimmomatic) Report</a></h4>

![alt text](images/TrimReport.png "Title")

<h4><a id="pAlign">Alignments</a></h4>

![alt text](images/PlantAlignedReads.png "Plant Aligned Reads")


<h4><a id="swAlign">Switchgrass Fungal alignment</a></h4>

![alt text](images/SwitchgrassFungalAlign.png "Title")


<h4><a id="aConc">Annotation Report</a></h4>
<p>Annotations were preformed using PROKKA annotations with gene identifiers mapped to KEGG Pathways. Exact annotation steps explained <a href='scripts/hpc_scripts/AnnotationPreprossesing.sh'>here</a>. The mapping and extraction of annotation counts can be found <a href='scripts/hpc_scripts/NGS_Mapping.sh'>here</a>.</p>

<h4><a id="aConc">Alignment &amp; Annotation Observations</a></h4>
<ol>
    <li>The percentage of plant reads sequenced is high thoughout the season and tapers off towards the end of the season. This may be do to senescence of plant cells.</li>
    <li>Alignment to the 6 selected fungal assemblies is low, but picks up during the warmer months. Overall, reads aligning to the combined fungal assembly were &lt;10% of the non-plant reads.</li>
    <li>The overlap between which contigs have both a prokaryotic and a eukaryotic annotation suggest either shared gene similarity or eukarotic contamination.</li>
</ol>

<h4>Is there a seasonal pattern of functional selection?</h4>